# 400.1 | Prompt Engineering For Developers

Notes and exercises based on the [ChatGPT Prompt Engineering For Developers](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/) course from DeepLearning.AI.

---

### 0. Validate Setup

Before you begin working on exercises in this section, make sure you are setup to work with the right LLM Providers by running the relevant code cells from the [LLM Setup](./400-00-llm-setup.ipynb) notebook.

---

### 1. Introduction 

This course was created when OpenAI was using version 0.28. In this notebook I hope to use the course for insights, but use it in conjunction with the OpenAI documentation to update the learnings for the latest OpenAI versions and practices. Quick study notes

1. LLM = Large Language Model. Understands Natural Language. Takes NLP inputs (prompts), returns NLP outputs (responses).
1. 2 types of LLM = Base LLM (predicts next word), Instruction Tuned LLM (follows instructions)
1. Base LLM = Default, also referred to as Foundation Models that are trained on large corpora of text data.
1. Instruction Tuned LLM = Fine-tuned version of base LLM using RLHF (Reinforcement Learning from Human Feedback) to follow instructions.
1. Generative AI Focus = Increasingly on Instruction Tuned LLMs, and using NLP to have AI execute tasks based on instruction prompts.

LLMs are smart in the sense of having a massive knowledge base and fast pattern-matching capabilities. But they lack the human cognitive ability to understand context, reason, and infer meaning. Instead, they are effectively reflecting the data they were trained on - which means instruction-tuned LLMs are only as effective as the clarity and specificity of instructions they are given.


The course has its own sandbox environment for practice, but I want to try my own experiments here. 
- Refer to the [Setup notebook](400-00-llm-setup.ipynb) for code snippets for chat completion
- Refer to the [OpenAI API documentation](https://platform.openai.com/docs/quickstart?context=python) for quickstarts
- Refer to the [OpenAI Cookbook](https://cookbook.openai.com/) for richer examples

---

### 2. Principle 1: Write Clear and Concise Instructions

#### 2.0 Helper Function

In [4]:
# Create an OpenAI client instance
from openai import OpenAI
client = OpenAI()

# Create a helper function to reuse across exercises
# So we can focus on prompt engineering more clearly
def get_completion(prompt, model="gpt-3.5-turbo"):
  messages = [{"role": "user", "content": prompt }]
  completion = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0, #degree of randomness
  )
  return completion.choices[0].message.content

# Prompting Principles
# 1. Write clear and specific instructions
# 2. Give the model time to "think"
# =====================================

#### 2.1 Tactic: Delimiters For Contextual Boundaries

Give model clear guidance on what it should focus on for the task. Delimiters can also reduce prompt injection attacks by guiding it to focus on the right context.

In [14]:

# Make sure cells are run in order to have client setup.
# The prompt (user or system message) defines delimiter.

text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
noise = f"""
Let's talk about amazing food that you can get when \
travel the world and explore different cultures. \
I love sushi in Japan and pasta in Italy. \
I love paella in Spain and tacos in Mexico. \
"""

# Craft Prompt
prompt = f"""
Summarize the text delimited by triple asterisks \ 
into 3 bullet points each with under 10 words.
{text}***{noise}***
"""

# Print Completion 
print(get_completion(prompt))

- Explore diverse cultures through food
- Enjoy sushi in Japan, pasta in Italy
- Try paella in Spain, tacos in Mexico


#### 2.2 Tactic: Ask For Structured Output

In [10]:
# Craft Prompt
prompt = f"""
Generate a list of three imaginary movies \ 
with a short teaser description, genre and movie rating. 
Provide them in JSON format with the following keys: 
movie_id, title, description, genre, rating.
"""

# Print Completion 
print(get_completion(prompt))

```json
[
    {
        "movie_id": 1,
        "title": "The Last Frontier",
        "description": "In a post-apocalyptic world, a group of survivors must navigate through dangerous territories to find a safe haven known as The Last Frontier.",
        "genre": "Action/Adventure",
        "rating": "PG-13"
    },
    {
        "movie_id": 2,
        "title": "Echoes of the Past",
        "description": "A detective uncovers a series of murders that seem to be connected to a dark secret from his own past, leading him on a dangerous journey to solve the mystery.",
        "genre": "Mystery/Thriller",
        "rating": "R"
    },
    {
        "movie_id": 3,
        "title": "Dreamscape",
        "description": "A young woman discovers she has the ability to enter people's dreams, but soon realizes that her gift comes with a dark side that threatens to consume her.",
        "genre": "Fantasy/Horror",
        "rating": "PG-13"
    }
]
```


#### 2.3 Tactic: Ask Model to Check Conditions Met


In [22]:
# Primary Content
good_text = f"""
To make a good almond cookie you need to buy a pack of almond flour \
and have 1 egg, 1 stick of butter and a cup of brown sugar on hand. \
Get a bowl and mix those ingredients with a pinch of salt and baking \
soda - but make sure you melt the butter first. Then wrap the dough \
and refrigerate for an hour. Preheat the oven to 350 degrees. Remove \
dough, flatten it out and cut out rounds. Bake for 10 minutes. \ 
Let them cool and enjoy your cookies!
"""
bad_text = f"""
It's a beautiful day outside. The sun is shining, the birds are singing. \
The flowers are blooming. On days like these, I want to bake cookies!
"""

# Craft Prompt
instruction = f"""
You will be provided with text delimited by four asterisks. \
If that texts contains a sequence of instructions rewrite in steps like: \

- First ... 
- Then ..
- Then ..
...
- Finally ..

Make sure each step has only 1 sentence. Break things down.
Use simple conversational language and be clear
If text has no instructions write \"Nothing to do!\"

****{good_text}{bad_text}****
"""

# Print Completion 
prompt = f"""
{instruction}****{good_text}****
"""
print(get_completion(prompt))
print("-------------")

# Print Completion 
prompt = f"""
{instruction}****{bad_text}****
"""
print(get_completion(prompt))
print("-------------")


First, buy almond flour, an egg, butter, and brown sugar. 
Then, mix those ingredients in a bowl with salt and baking soda after melting the butter. 
Then, wrap the dough and refrigerate for an hour. 
Next, preheat the oven to 350 degrees. 
Then, remove the dough, flatten it, and cut out rounds. 
Finally, bake for 10 minutes, let them cool, and enjoy your cookies!
-------------
Nothing to do!
-------------


#### 2.4 Tactic: Few Shot Prompting (by examples)

Give examples of what you want model to do and let is figure out what to do, then give it a task to do.


In [23]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends in the \ 
strongest winds but never breaks; it is the \ 
ability to bounce back from adversity and \ 
continue to grow and thrive despite challenges.


---

### 3. Principle 2: Give the Model Time To Think


#### 3.1 Tactic: Specify steps required (chain of reasoning)

Give the model a series of steps it should take before it returns the response. Slow the model down so it puts more time into the computation instead of returning an immediate result. The added effort will improve the quality

In [31]:

# example 1: Give it time to think
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""

prompt1 = f"""
Perform the following actions: 
1 - Summarize text delimited by triple asterisks, in one SHORT sentence \
2 - Translate summary into Spanish.
3 - List each name in the Spanish summary.
4 - Output a json object that contains the following \
keys: spanish_summary, num_names.

Separate answers for each step with "----" lines.

Text:
***{text}***
"""
print("Completion 1:")
print(get_completion(prompt1))

# example 2: Specify Format
prompt2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
print("Completion 2:")
print(get_completion(prompt2))

Completion 1:


----
1 - Jack and Jill go on a quest to fetch water, but misfortune strikes as Jack trips and tumbles down a hill with Jill following suit, yet they return home with comforting embraces and continue exploring with delight.
2 - Jack y Jill van en una búsqueda para traer agua, pero la desgracia golpea cuando Jack tropieza y cae por una colina con Jill siguiendo el ejemplo, sin embargo, regresan a casa con abrazos reconfortantes y continúan explorando con alegría.
3 - Jack, Jill
4 - {
    "spanish_summary": "Jack y Jill van en una búsqueda para traer agua, pero la desgracia golpea cuando Jack tropieza y cae por una colina con Jill siguiendo el ejemplo, sin embargo, regresan a casa con abrazos reconfortantes y continúan explorando con alegría.",
    "num_names": 2
} 
----
Completion 2:
Summary: Jack and Jill, siblings, go on a quest to fetch water but encounter misfortune on the way back.

Translation: Jack et Jill, frère et sœur, partent en quête d'eau mais rencontrent un malheur sur le c

#### 3.2 Tactic: Instruct Model To Work it out (chain of thought)

Instead of telling model what to do (give it a chain of reasoning) ask it to tell us how it would solve the problem (chain of thought). This slows model down and makes it think before rushing to a conclusion (otherwise it may get the wrong response). Try it.

In [38]:
problem = f"""
Question:
```
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.
```
"""

student_solution = """
Student's Solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
"""

prompt1 = f"""
Determine if the student's solution is correct or not.
{problem}
{student_solution}
"""

prompt2= f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```
{problem}
{student_solution}
Actual solution:
"""


# example 1: No chain of thought
print("Quick Response:")
print(get_completion(prompt1))

# example 2: No chain of thought
print("-------------------")
print("Measured Response:")
print(get_completion(prompt2))

Quick Response:


The student's solution is correct. The total cost for the first year of operations as a function of the number of square feet is indeed 450x + 100,000.
-------------------
Measured Response:
Let x be the size of the installation in square feet.

Costs:
1. Land cost: $100 * x
2. Solar panel cost: $250 * x
3. Maintenance cost: $100,000 + $10 * x

Total cost: $100 * x + $250 * x + $100,000 + $10 * x = $360 * x + $100,000

The total cost for the first year of operations as a function of the number of square feet is $360x + $100,000.
```

Is the student's solution the same as actual solution just calculated:
```
No
```

Student grade:
```
incorrect
```


#### 3.3 Model Limitations: Hallucinations

Fabricated information that the model believes to be true. This can happen when the model is asked to generate information that is not in the training data - and it confabulates an answer. It makes responds sound plausible. One way to reduce this is to ask the model to find relevant information in the text and answer questions based on the extracted concepts and find citable references.

In [39]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush designed to provide a superior cleaning experience. It features ultra-thin bristles that are gentle on the gums and teeth, while still effectively removing plaque and debris. The toothbrush also has a built-in timer and pressure sensor to help ensure you are brushing for the recommended two minutes and not applying too much pressure.

The smart toothbrush connects to a mobile app via Bluetooth, allowing you to track your brushing habits and receive personalized recommendations for improving your oral hygiene routine. The app also provides reminders to replace your brush head when it is time for a new one.

The AeroGlide UltraSlim Smart Toothbrush is made with antimicrobial materials to prevent bacteria growth and is designed to be long-lasting and environmentally friendly. It is rechargeable and comes with a sleek charging dock for easy storage and charging. Overall, this toothbrush offers a modern and effective

---

### 4. Iterative Prompt Development

Prompt Engineering is more art than science. Similar to data science (MLOps) lifecycles, we have { idea generation, implementation, evaluation } cycles of iterative development. 

The Prompt Engineering process is now:
 - Try something => write a prompt to get a task done
 - Evaluate response => check if response is what you wanted with a sample input
 - Iterate prompt => analyze why response was flawed and clarify task in prompt
 - Repeat evaluate/iterate cycle => get prompt to acceptable state with ONE sample
 - Extend evaluate/iterate cycle => repeat this with more examples to refine prompt for fairness and consistency

The key is to have a an **evaluation** process to let us see if changes we make to the prompt are having desired changes to completion quality. 
 - Evaluation can start as a manual interactive process then move towards more evaluation at scale
 - First = manual evaluation with one target example => iterate till prompt gives me desired response for that example 👉🏽 analogy = teacher grades a student paper (defines grading rubric)
 - Next = manual evaluation with more target examples => check if refined prompt works with broader set of examples 👉🏽 analogy = teacher grades full class papers (refines grading rubric)
 - Next = AI-assisted evaluation with more examples => train another AI to check the responses for you 👉🏽 analogy = teacher asks TA to grade class paper (provides grading rubric)
 - Next = AI-assisted evaluation at scale => use AI to evaluate responses at scale in CI/CD pipelines 👉🏽 analogy = school asks TA org to grade all papers for all teachers  (provides grading rubric for all exams)

Responsible AI now means we need to make sure teacher and rubric were fair (unbiased) and TA/Org were consistent and accountable in grading with the rubric.


##### 4.1 Create Initial Prompt (Retail Catalog)

"Generate a marketing product description from a fact sheet"

In [49]:
# Catalog Item
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [50]:
# Initial Prompt
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our stylish and versatile mid-century inspired office chair, perfect for both home and business settings. This chair is part of a beautiful family of office furniture that includes filing cabinets, desks, bookcases, meeting tables, and more.

Customize your chair with several options of shell color and base finishes to suit your personal style. Choose between plastic back and front upholstery or full upholstery in a variety of fabric and leather options. The base finish options include stainless steel, matte black, gloss white, or chrome. You can also choose to have armrests or go for a sleek armless design.

Constructed with a 5-wheel plastic coated aluminum base and featuring a pneumatic chair adjust for easy raise/lower action, this chair is both durable and functional. The dimensions of the chair are as follows: width 53 cm, depth 51 cm, height 80 cm, seat height 44 cm, and seat depth 41 cm.

Personalize your chair even further with options such as soft or hard-floor ca

##### 4.2 Refine Initial Prompt (Response Length)
"Issue: The text was too long. Let's adjust that"

In [53]:
# Refined Prompt
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

USE AT MOST 50 WORDS

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile and stylish mid-century office chair, available in a variety of colors and finishes to suit any space. With adjustable height and comfortable upholstery options, this chair is perfect for both home and business use. Made with quality materials from Italy, this chair is built to last.


##### 4.3 Refine Iterated Prompt (Response Quality)

"Issue: The text missed key details. Let's correct that"

In [54]:
# Refine Prompt
# Add request to refine focus for response
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile and stylish office chair, part of a mid-century inspired furniture collection. Choose from a variety of shell colors and base finishes to suit your space. Constructed with a durable aluminum base and high-density foam seat for comfort. Perfect for home or business use. 

Product IDs: SWC-100, SWC-110


##### 4.4 Refine Iterated Prompt (Response Format)

"Isse: I need this in HTML format for web page. Let's fix that"

In [55]:
# Refine Prompt
# Get the description to add specific section with details in a table
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

```html
<div>
    <p>This mid-century inspired office chair is a stylish and versatile addition to any workspace. Constructed with a durable 5-wheel plastic coated aluminum base, this chair offers both style and functionality. The seat is made with high-density HD36 foam for maximum comfort and support. The chair is available in a variety of shell colors and base finishes to suit any decor. Whether for home or business use, this chair is a perfect choice for any setting.</p>
    
    <p>Product IDs: SWC-100, SWC-110</p>
    
    <h2>Product Dimensions</h2>
    <table>
        <tr>
            <td>WIDTH</td>
            <td>20.87"</td>
        </tr>
        <tr>
            <td>DEPTH</td>
            <td>20.08"</td>
        </tr>
        <tr>
            <td>HEIGHT</td>
            <td>31.50"</td>
        </tr>
        <tr>
            <td>SEAT HEIGHT</td>
            <td>17.32"</td>
        </tr>
        <tr>
            <td>SEAT DEPTH</td>
            <td>16.14"</td>
        </tr>
   

In [56]:
# Hard to review the markdown manually - let's use the code cell to format it the way the web page would render it
from IPython.display import display, HTML
display(HTML(response))

WIDTH,"20.87"""
DEPTH,"20.08"""
HEIGHT,"31.50"""
SEAT HEIGHT,"17.32"""
SEAT DEPTH,"16.14"""


That looks good - let's ship it!

---

### 5. Summarizing

This exercise explores how prompts can be engineered to improve the quality of a **summarization task** that takes a large volume of text ("primary content") and has the model generate a concise summary of the content ("response") by providing the relevant instructions ("prompt")

In [61]:
# Primary Content: Single Product Review
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

##### 5.1 First Prompt: Summarize Review

In [64]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


Summary: 
The panda plush toy is soft, cute, and has a friendly face, making it a hit with the reviewer's daughter. However, the reviewer feels that it is a bit small for the price paid and suggests that there may be larger options available for the same price. Despite this, the reviewer was pleased with the early delivery and had the chance to enjoy the toy before giving it to their daughter.


#### 5.2 Refine Prompt: Specify Length

In [65]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

```
Cute panda plush toy loved by daughter, soft and friendly-looking. Smaller than expected for the price, but arrived early.
```


#### 5.3 Refine Prompt: Create Translations

In [66]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Then translate that review into Spanish, French, Italian and German.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

Summary: "Cute panda plush toy, soft and loved by daughter. Small for price, but arrived early."

Spanish: "Peluche de panda lindo, suave y amado por mi hija. Pequeño para el precio, pero llegó temprano."
French: "Peluche panda mignon, doux et aimé par ma fille. Petit pour le prix, mais est arrivé tôt."
Italian: "Peluche panda carino, morbido e amato da mia figlia. Piccolo per il prezzo, ma è arrivato presto."
German: "Niedliches Panda-Plüschtier, weich und von meiner Tochter geliebt. Klein für den Preis, aber kam früh an."


#### 5.4 Refine Prompt: Refine Focus On Shipping

In [70]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Print the English review first.
Then translate that review into Spanish, French, Italian and German.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

English:
"The panda plush toy arrived a day earlier than expected. It's soft, cute, and my daughter loves it. However, it's a bit small for the price."

Spanish:
"El peluche de panda llegó un día antes de lo esperado. Es suave, lindo y a mi hija le encanta. Sin embargo, es un poco pequeño para el precio."

French:
"Le jouet en peluche panda est arrivé un jour plus tôt que prévu. Il est doux, mignon et ma fille l'adore. Cependant, il est un peu petit pour le prix."

Italian:
"Il peluche panda è arrivato un giorno prima del previsto. È morbido, carino e mia figlia lo adora. Tuttavia, è un po' piccolo per il prezzo."

German:
"Das Panda-Plüschtier kam einen Tag früher als erwartet an. Es ist weich, niedlich und meine Tochter liebt es. Allerdings ist es etwas klein für den Preis."


#### 5.5 Refine Prompt: Refine Focus On Price

In [71]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words,  and focusing on any aspects \
that are relevant to the price and perceived value. 

Print the English review first.
Then translate that review into Spanish, French, Italian and German.

Review: ```{prod_review}```
"""

#### 5.6 Refine Prompt: Use Extract not Summarize

In [73]:
prompt = f"""
Your task is to extract relevant information from \ 
a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

Feedback: The product arrived a day earlier than expected, allowing the customer to play with it before giving it as a gift.


#### 5.7 Refine Prompt: Validate with Multiple Reviews

In [74]:

review_1 = prod_review 

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]

In [77]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Start each summary with a single word that describes the core sentiment of the review.

    Review: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

0 Positive: Adorable panda plush toy, soft and cute. Small for price, but arrived early. 

1 Positive: Lamp with storage, fast delivery, excellent customer service, easy assembly, missing part quickly resolved. Great company. 

2 Positive: Impressive battery life, small head, good deal for $50, generic replacement heads available, teeth feel clean. 

3 Mixed: Price fluctuations, quality concerns, motor issues, and tips for usage. Brand loyalty emphasized. Delivery prompt. 



---

### 6. Inferring

Here the model takes input and does some processing to _infer information_ before it returns the response. The main difference is that I am replacing traditional "machine learning" (train model to do X) with "generative AI" (ask model to do X) using natural language.

In [78]:
# Product review text (primary content)
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

#### 6.1 First Prompt: Infer sentiment for Review

In [79]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive. The reviewer is satisfied with the lamp, the customer service, and the company in general.


#### 6.2 Refine: Specify acceptable values

In [80]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Positive


#### 6.3 Refine: Identify types of emotion

In [81]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, grateful, impressed, content


#### 6.4 Refine: Look for specific emotion

In [82]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

prompt = f"""
Is the writer of the following review expressing anger, joy, sadness or fear?\
The review is delimited with triple backticks. \
Give your answer as either yes or no for each emotion.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No
Anger: No
Joy: Yes
Sadness: No
Fear: No


#### 6.5 Refine: Extract information with format

In [83]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


#### 6.6 Refine: Do multiple tasks in one prompt

In [91]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Sentiment": "positive",
    "Anger": false,
    "Item": "lamp",
    "Brand": "Lumina"
}


#### 6.7 First Prompt: Infer Topics from Story

In [92]:
# Story
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

In [93]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items SEPARATED BY COMMAS.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

1. Survey
2. Job satisfaction
3. NASA
4. Social Security Administration
5. Government pledge


##### 6.8 Refine: Specify separator, enumeration

In [ ]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""

---

### 7. Transforming

---

### 8. Expanding

---

### 9. Chatbot

---

### 10. Conclusion